## **闭包**
### **闭包的构成条件**
- 有嵌套：在函数嵌套（函数里面再定义函数）的前提下
- 有变量：内部函数使用了外部函数的变量（参数）
- 有引用：外部函数的返回值是内部函数的引用

In [8]:
def outer(a):
    def inner(b):
        nonlocal a
        # global a
        a = a + b
        return a
    return inner

print(outer(1)(2))
inner = outer(10)
print(inner(1))
print(inner(1))
print(inner(1))

3
11
12
13


In [18]:
def outer1(a):
    b = 7
    def inner():
        c = 11
        return a + b + c
    return inner

inner1 = outer1(10)
print(inner1.__closure__)
print(inner1.__closure__[0].cell_contents)
print(inner1.__closure__[1].cell_contents)

(<cell at 0x0000023AC4A6BE20: int object at 0x00007FFF4A1554C8>, <cell at 0x0000023AC4D1B910: int object at 0x00007FFF4A155468>)
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'cell_contents']
10
7


In [19]:
def outer():
    funcs = []
    for k in range(3):
        def inner():
            return k * k
        funcs.append(inner)
    return funcs

f1, f2, f3 = outer()
# 这里的f1, f2, f3都是闭包, 本质上都是inner函数, 且使用了outer函数的变量k,
# outer函数在结束时, 将变量k保存到内部函数的__closure__属性中,而k最后为2
print(f1.__closure__[0].cell_contents)
print(f2.__closure__[0].cell_contents)
print(f3.__closure__[0].cell_contents)
print(f1())
print(f2())
print(f3())

2
2
2
4
4
4


## **装饰器**

### **不带参数的函数装饰器**

In [27]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__}的执行时间：{end - start}s')
        return result
    return wrapper

def time_test_v1():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
timer(time_test_v1)()

@timer
def time_test_v2():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
time_test_v2()

371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v1的执行时间：0.00010919570922851562s
371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v2的执行时间：9.1552734375e-05s


### **带参数的函数装饰器**

In [28]:
import time

def timer(username= 'hermesfuxi'):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f'{username}的{func.__name__}函数执行时间：{end - start}s')
            return result
        return wrapper
    return decorator

def time_test_v1():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
timer('hermes')(time_test_v1)()

@timer('hermes')
def time_test_v2():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
time_test_v2()

371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
hermes的time_test_v1函数执行时间：0.00010991096496582031s
371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
hermes的time_test_v2函数执行时间：6.246566772460938e-05s


### **不带参数的类装饰器**

In [30]:
import time

class Timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        end = time.time()
        print(f'{self.func.__name__}函数执行时间：{end - start}s')
        return result

def time_test_v1():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
Timer(time_test_v1)()

@Timer
def time_test_v2():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)
time_test_v2()

371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v1函数执行时间：0.00015497207641601562s
371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v2函数执行时间：5.054473876953125e-05s


### **带参数的类装饰器**

In [6]:
import time

class Timer:
    def __init__(self, name):
        self.name = name

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f'{func.__name__}函数执行时间：{end - start}s')
            return result
        return wrapper

def time_test_v1():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)

Timer('hermes')(time_test_v1)()

@Timer('hermes')
def time_test_v2():
    sum = 0
    for i in range(100):
        sum += i ** i
    print(sum)

time_test_v2()

371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v1函数执行时间：0.00011014938354492188s
371115746176445351701210713361941528546861949073514542015172437236580034634746971244943788132460150776779198800002366059871900041784732217539059306483834977865973576751345853385981719448969027641921
time_test_v2函数执行时间：4.863739013671875e-05s
